In [ ]:
import os

directory = "../data/raw"

if not directory:
    directory = input("Input raw dataset directory:")

files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

count = 1
if files:
    curr = files[0].split("_")[0]
else:
    print("No files found in the directory.")
    exit()

for file in files:
    file_split = file.split("_")
    if file_split[0] != curr:
        curr = file_split[0]
        count += 1
    
    new_file_name = "_".join(file_split[1:])
    new_file_name = str(count).zfill(3) + "_" + new_file_name
    
    old_path = os.path.join(directory, file)
    new_path = os.path.join(directory, new_file_name)
    
    os.rename(old_path, new_path)
    print(f"Successfully changed file name from ({file}) to ({new_file_name})")

In [ ]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def skeletonize(img):
    """Alternative skeletonization implementation without ximgproc"""
    skel = np.zeros(img.shape, np.uint8)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    while True:
        open_img = cv2.morphologyEx(img, cv2.MORPH_OPEN, element)
        temp = cv2.subtract(img, open_img)
        eroded = cv2.erode(img, element)
        skel = cv2.bitwise_or(skel, temp)
        img = eroded.copy()
        if cv2.countNonZero(img) == 0:
            break
    return skel

def process_fingerprint(image_path, output_path, target_size=None, upscale_factor=1.0):
    # Read image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")
    
    # Resize if needed
    if target_size:
        image = cv2.resize(image, target_size, interpolation=cv2.INTER_LANCZOS4)
    elif upscale_factor != 1.0:
        h, w = image.shape
        image = cv2.resize(image, (int(w*upscale_factor), int(h*upscale_factor)), 
                         interpolation=cv2.INTER_LANCZOS4)
    
    # 1. Contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(image)
    
    # 2. Noise reduction
    denoised = cv2.bilateralFilter(enhanced, 9, 75, 75)
    
    # 3. Adaptive thresholding
    thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                cv2.THRESH_BINARY_INV, 21, 7)
    
    # 4. Morphological operations
    kernel = np.ones((3,3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel, iterations=1)
    
    # 5. Skeletonization (using alternative method)
    skeleton = skeletonize(morph)
    
    # 6. Final inversion and saving
    result = cv2.bitwise_not(skeleton)
    cv2.imwrite(output_path, result)

def batch_process_fingerprints(input_folder, output_folder, upscale=False):
    """
    Process all fingerprints in a folder
    """
    os.makedirs(output_folder, exist_ok=True)
    files = [f for f in os.listdir(input_folder) if f.lower().endswith('.tif')]
    
    for filename in tqdm(files, desc="Processing Fingerprints"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        try:
            if upscale:
                process_fingerprint(input_path, output_path, upscale_factor=2.0)
            else:
                process_fingerprint(input_path, output_path)
        except Exception as e:
            print(f"\nError processing {filename}: {str(e)}")
            continue

if __name__ == "__main__":
    INPUT_FOLDER = "../data/raw"
    OUTPUT_FOLDER = "../data/processed"
    UPSCALE_IMAGES = True  # Set to True for 2x upscaling
    
    print("Starting fingerprint processing...")
    batch_process_fingerprints(INPUT_FOLDER, OUTPUT_FOLDER, upscale=UPSCALE_IMAGES)
    print("\nProcessing completed successfully!")

In [ ]:
import os
import shutil
import random
from tqdm import tqdm

# Set random seed for reproducibility
random.seed(42)  # Ensures the same split every time you run the code

# Define paths
dataset_folder = "../data/processed"
train_folder = "../data/final/train"
test_folder = "../data/final/test"

# Create train and test folders
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Get all image files and shuffle them
all_files = [f for f in os.listdir(dataset_folder) if f.endswith('.tif')]
random.shuffle(all_files)  # Shuffle to ensure random distribution

# Calculate split index (80% train, 20% test)
split_idx = int(0.8 * len(all_files))
train_files = all_files[:split_idx]  # First 80% for training
test_files = all_files[split_idx:]   # Remaining 20% for testing

print(f"Found {len(all_files)} total fingerprint images")
print(f"- Training images: {len(train_files)} (80%)")
print(f"- Test images: {len(test_files)} (20%)")

# Function to copy files
def copy_files(file_list, destination_folder):
    for file in tqdm(file_list, desc=f"Copying to {os.path.basename(destination_folder)}"):
        src = os.path.join(dataset_folder, file)
        dest = os.path.join(destination_folder, file)
        shutil.copy2(src, dest)

# Copy files to respective folders
copy_files(train_files, train_folder)
copy_files(test_files, test_folder)

print("\nDataset organization completed:")
print(f"- Training set: {len(train_files)} images (copied to {train_folder})")
print(f"- Test set: {len(test_files)} images (copied to {test_folder})")
print(f"Original files remain intact in {dataset_folder}")

In [ ]:
import cv2
import numpy as np
import os
import random
from tqdm import tqdm

# Path configuration
test_dir = "../data/final/test"

def apply_block_damage_smart(image, block_size=80, num_blocks=7):
    """Apply white block damage only on fingerprint area."""
    damaged = np.copy(image)
    height, width = image.shape
    mask = image < 250
    ys, xs = np.where(mask)

    if len(xs) == 0:
        return damaged

    for _ in range(num_blocks):
        idx = random.randint(0, len(xs) - 1)
        x_center, y_center = xs[idx], ys[idx]
        x1 = max(0, x_center - block_size // 2)
        y1 = max(0, y_center - block_size // 2)
        x2 = min(width, x1 + block_size)
        y2 = min(height, y1 + block_size)
        damaged[y1:y2, x1:x2] = 255
    return damaged

def apply_blur_damage(image, block_size=60, num_blocks=5):
    damaged = np.copy(image)
    height, width = image.shape

    for _ in range(num_blocks):
        x = random.randint(0, width - block_size)
        y = random.randint(0, height - block_size)
        roi = damaged[y:y+block_size, x:x+block_size]
        blurred = cv2.GaussianBlur(roi, (11, 11), 0)
        damaged[y:y+block_size, x:x+block_size] = blurred
    return damaged

def apply_elliptical_noise(image, num_ellipses=5):
    damaged = np.copy(image)
    height, width = image.shape

    for _ in range(num_ellipses):
        center = (
            random.randint(0, width),
            random.randint(0, height)
        )
        axes = (
            random.randint(20, 60),
            random.randint(10, 30)
        )
        angle = random.randint(0, 180)
        cv2.ellipse(damaged, center, axes, angle, 0, 360, 255, -1)
    return damaged

def apply_combined_damage(image):
    """Apply all three types of damage sequentially."""
    image = apply_block_damage_smart(image)
    image = apply_blur_damage(image)
    image = apply_elliptical_noise(image)
    return image

# Get all test images (.tif files)
test_images = [f for f in os.listdir(test_dir) if f.endswith(".tif")]
print(f"Found {len(test_images)} test images to process")

# Process and overwrite originals
for filename in tqdm(test_images, desc="Applying combined damage"):
    image_path = os.path.join(test_dir, filename)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if image is not None:
        damaged_image = apply_combined_damage(image)
        cv2.imwrite(image_path, damaged_image)

print(f"\nOverwritten {len(test_images)} images with combined damage in {test_dir}")

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(len(base_model.layers))